In [45]:
import pandas as pd
import json

# Load your dataset (modify the filename accordingly)
file_path = "Data4Good_Arolsen_Archives_50k.csv"  # Change to your actual file path
column_name = "Geo Location"    # Change to your actual column name

# Read the dataset
df = pd.read_csv(file_path)


# Extract the JSON column
geo_data = df[column_name].tolist()

# Save to a JSON file
output_file = "geo_data.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(geo_data, f, ensure_ascii=False, indent=4)

print(f"JSON data saved to {output_file}")


JSON data saved to geo_data.json


In [67]:
import json
import pandas as pd
import re
from datetime import datetime

# Load the general dataset with dates information
general_data_file = "Data4Good_Arolsen_Archives_50k.csv"  # Path to the general dataset
data = pd.read_csv(general_data_file)

# Load JSON data from the file
input_file = "geo_data.json"
with open(input_file, 'r', encoding='utf-8') as f:
    geo_data = json.load(f)

# Define functions
def create_time_intervals(birthdate, dates):
    if not dates:  # If no dates are available
        return []
    
    # Filter valid dates
    valid_dates = [d for d in dates if pd.notnull(d)]
    
    if pd.notnull(birthdate):
        all_dates = [birthdate] + sorted(valid_dates)
    else:
        all_dates = sorted(valid_dates)
    
    return all_dates  # Return all valid dates

def extract_dates(text):
    if pd.isnull(text):
        return []
    
    # Pattern to capture dates
    date_pattern = r'(?<!\d)(\d{1,2}[./]\d{1,2}[./]\d{2,4}|\d{1,2}[./]\d{2}|\d{4})(?!\d)'
    
    raw_dates = re.findall(date_pattern, text)
    
    possible_formats = ['%d.%m.%Y', '%d.%m.%y', '%m.%Y', '%m.%y', '%Y', '%m/%Y', '%m/%y']
    
    parsed_dates = []
    for date_str in raw_dates:
        for fmt in possible_formats:
            try:
                parsed_date = datetime.strptime(date_str, fmt)
                
                # Correct year if it's in the future
                if parsed_date.year > datetime.now().year:
                    parsed_date = parsed_date.replace(year=parsed_date.year - 100)
                
                parsed_dates.append(parsed_date.date())
                break  # Exit once parsed
            except ValueError:
                continue
    
    return parsed_dates

# Prepare dataset transformation
rows = []

for record_id, record_str in enumerate(geo_data, start=1):
    try:
        if not isinstance(record_str, str):
            record_str = "{}"  # Replace invalid values with empty JSON
        
        cleaned_str = record_str.strip('"').replace('""', '"')
        record = json.loads(cleaned_str)

        paths = record.get("paths", [])
        markers = {marker.get("label", "Unknown"): marker.get("type", "Unknown") for marker in record.get("markers", [])}
        
        # Get Middle value from the general dataset for extracting dates
        middle_value = data.loc[record_id - 1, 'Middle'] if record_id - 1 < len(data) else None
        extracted_dates = extract_dates(middle_value)
        
        # Get Birthdate from the general dataset
        birthdate_str = data.loc[record_id - 1, 'Birthdate (Geb)'] if record_id - 1 < len(data) else None
        birthdate = pd.to_datetime(birthdate_str, format='%d/%m/%Y', errors='coerce') if birthdate_str else None

        # Get the list of all valid dates (birthdate + extracted dates)
        all_dates = create_time_intervals(birthdate, extracted_dates)

        for index, path in enumerate(paths):
            origin = path.get("fromLabel", "Unknown")
            dest = path.get("toLabel", "Unknown")
            type_value = markers.get(origin, "Unknown")

            # Create the interval for the current index
            if index < len(all_dates) - 1:
                start_date = all_dates[index].strftime('%d/%m/%Y')
                end_date = all_dates[index + 1].strftime('%d/%m/%Y')
                interval = f"{start_date} - {end_date}"
            else:
                interval = "Unknown - Unknown"  # Default for last index or if there's no valid date

            rows.append({
                "ID": record_id,
                "Origin": origin,
                "Dest": dest,
                "Index": index,
                "Type": type_value,
                "Interval": interval
            })
        
        if not paths:
            rows.append({
                "ID": record_id,
                "Origin": "Unknown",
                "Dest": "Unknown",
                "Index": -1,
                "Type": "Unknown",
                "Interval": "Unknown - Unknown"
            })

    except Exception as e:
        print(f"Error processing record {record_id}: {e}")
        continue

# Convert to DataFrame
df = pd.DataFrame(rows)

# Save to CSV
output_file = "transformed_dataset_with_intervals.csv"
df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Structured data with intervals saved to {output_file}")


Structured data with intervals saved to transformed_dataset_with_intervals.csv


,ID,Origin,Dest,Index,Type,Date_Start,Date_End


In [ ]:
# Load the transformed dataset from the CSV file
df = pd.read_csv("transformed_dataset.csv")
